<br/>
<p style="text-align:center;font-weight:bold;font-size:400%">Dynamic Programming</p>

# Introduction

This notebook demonstrates implementation of algorithms presented in Lecture 3 of UCL RL course by David Silver.

Algorithms:
* Iterative Policy Evaluation
* Policy Iteration
* Value Iteration

Notes:
* As OpenAI gym doesn't have environment corresponding to gridworld used in lectures. We use FrozenLake-v0 instead

Sources:
* UCL Course on RL: http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html
  * Lecture 3 pdf: http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/DP.pdf
  * Lecture 3 vid: https://www.youtube.com/watch?v=Nd1-UUMVfz4

In [1]:
import numpy as np
import gym

import pdb

np.set_printoptions(linewidth=115)  # nice printing of large arrays

# Frozen Lake

Documentaiton for FrozenLake-v0: https://gym.openai.com/envs/FrozenLake-v0/

Frozen Lake is 4x4 grid:
<img src='frozenlake.png'>
Note on actions:
* environment is 'slippery' - choosing to go 'North' will result with 1/3 probability of moving West/North/East each.
* external walls are 'bouncy' - if agent to get off the grid, it remains in current state instead
* above result in optimal policy which prioritizes avoiding holes over getting to target

Let's make an environment

In [2]:
env = gym.make('FrozenLake-v0')
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


Create random policy

In [3]:
nb_states = env.env.nS        # number of possible states
nb_actions = env.env.nA       # number of actions from each state

Environment transition probabilities and rewards are stored in array __env.env.P__, e.g. from state 6 go west (action 0)

In [4]:
env.env.P[6][0]

[(0.3333333333333333, 2, 0.0, False),
 (0.3333333333333333, 5, 0.0, True),
 (0.3333333333333333, 10, 0.0, False)]

* given starting state __6__ and action __0__ (West), there is __33%__ chance ending in state __2__, with reward __0.0__, non-terminal
* given starting state __6__ and action __0__ (West), there is __33%__ chance ending in state __5__, with reward __0.0__, terminal (hole)
* given starting state __6__ and action __0__ (West), there is __33%__ chance ending in state __10__, with reward __0.0__, non-terminal

# Iterative Policy Evaluation

We will implement algorithm as follows:
* Initialize all state values to zero: $\mathbf{v \leftarrow 0}$
* Flatten Markov Decision Process to Markov Reward Process, i.e. assume policy is part of environment
* Iterate n times:
  * update all state values using Bellman Expectation Equation:
  
<img src="eq_policy_eval.png">

Where:
* $\mathbf{v}^{k+1}$ is vector of state-values $v(s)$ at iteration k+1
* $\mathbf{R}^\pi$ is vector of expected rewards for each state
* $\gamma$ is a discount factor, here equals 1.0
* $\mathbf{P}^\pi$ is transition probability matrix under our policy $\pi$ with shape [nb_states, nb_states]
* $\mathbf{v}^k$ is vector of state-values $v(s)$ at iteration k

What we know:
* $\pi(a|s)$ - policy - probability of choosing action a, given state s - defined by us
* $P_{s,s'}^a$ - transition probability from state-action s,a to state s' (this is __33%__ above)
* $R_{s,s'}^a$ - expected reward on transition from state-action s,a to state s' (this is __0.0__ above)


First we will calculate transition probability from state s to s' as follows
<img src="eq_P_pi.png">
Where:
* $P_{s,s'}^\pi$ - transition probability from state s to s'

Then we use $P_{s,s'}^\pi$ to form transition probability matrix $\mathbf{P}^\pi$
<img src="eq_P_matrix.png">

Second we calculate expected reward on transition from state-action pair

<img src="eq_R_ssa.png">
Where:
* $R_s^a$ - expected reward on transition from state-action s,a to any next state (weighted sum over all next-states)

Now we can calculate expected reward from state s (not state-action!) to any next state
<img src="eq_R_pi.png">
Where:
* $R_s^\pi$ - expected reward on transition from state s to any state s' (weighted sum over all actions)

Helper function to calculate both $\mathbf{P}^\pi$ and $\mathbf{R}^\pi$

In [9]:
def flatten_mdp(policy, model):
    P_pi = np.zeros([nb_states, nb_states])  # transition probability matrix (s) to (s'), shape: [nb_states, nb_states]
    R_pi = np.zeros([nb_states])             # expected reward from state (s) to any next state, shape: [nb_states]
    for s in range(nb_states):
        for a in range(nb_actions):
            for p_, s_, r_, _ in model[s][a]:
                # p_ - transition probability from (s,a) to (s')
                # s_ - next state (s')
                # r_ - reward on transition from (s,a) to (s')
                P_pi[s, s_] += policy[s,a] * p_   # transition probability (s) -> (s')
                Rsa = p_ * r_                     # expected reward for transition from (s,a) to any next state
                R_pi[s] += policy[s,a] * Rsa      # expected reward for transition from (s) to any next state
    assert np.alltrue(np.sum(P_pi, axis=-1)==np.ones([nb_states]))  # rows should sum to 1
    return P_pi, R_pi

Lets do policy evaluation for random policy

In [82]:
policy = np.ones([nb_states, nb_actions]) / nb_actions  # random policy, 25% each action
P_pi, R_pi = flatten_mdp(policy, model=env.env.P)

In [83]:
print(P_pi)

[[0.5  0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.5  0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.   0.   0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.25 0.25 0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
 [0.

In [84]:
print(R_pi)

[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]


Perform 100 steps of iterative policy evaluation according to equation
<img src="eq_policy_eval.png">

In [85]:
lmbda = 1.0  # discount

V_pi = np.zeros([nb_states])
for i in range(100):
    V_pi = R_pi + lmbda * P_pi @ V_pi
print(V_pi.reshape([4, -1]).round(3))

[[0.014 0.012 0.021 0.01 ]
 [0.016 0.    0.041 0.   ]
 [0.035 0.088 0.142 0.   ]
 [0.    0.176 0.439 0.   ]]


Correct output:
```
[[0.014 0.012 0.021 0.01 ]
 [0.016 0.    0.041 0.   ]
 [0.035 0.088 0.142 0.   ]
 [0.    0.176 0.439 0.   ]]
```

# Policy Iteration

Function to calculate greedy policy based on current policy state-values and environment model
<img src="diag_Q_pi.png"/>
<img src="eq_Q_pi.png"/>

In [19]:
def calc_Q_pi(V_pi, model, lmbda):
    Q_pi=np.zeros([nb_states, nb_actions])
    for s in range(nb_states):
        for a in range(nb_actions):
            for p_, s_, r_, _ in model[s][a]:
                Pssa = p_       # probability going from s,a -> s_
                Rsa = p_ * r_   # expected reward for transition s,a -> s_
                Vs_ = V_pi[s_]  # state-value of s_
                Q_pi[s, a] += Rsa + lmbda * Pssa * Vs_
    return Q_pi

In [20]:
Q_pi = calc_Q_pi(V_pi, env.env.P, lmbda)   # calc Q_pi for values from policy evalutaion section above
print(Q_pi.round(3))

[[0.015 0.014 0.014 0.013]
 [0.009 0.012 0.011 0.016]
 [0.024 0.021 0.024 0.014]
 [0.01  0.01  0.007 0.014]
 [0.022 0.017 0.016 0.01 ]
 [0.    0.    0.    0.   ]
 [0.054 0.047 0.054 0.007]
 [0.    0.    0.    0.   ]
 [0.017 0.041 0.035 0.046]
 [0.07  0.118 0.106 0.059]
 [0.189 0.176 0.16  0.043]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.088 0.205 0.234 0.176]
 [0.252 0.538 0.527 0.439]
 [0.    0.    0.    0.   ]]


Correct output:
```
[[0.015 0.014 0.014 0.013]
 [0.009 0.012 0.011 0.016]
 [0.024 0.021 0.024 0.014]
 [0.01  0.01  0.007 0.014]
 [0.022 0.017 0.016 0.01 ]
 [0.    0.    0.    0.   ]
 [0.054 0.047 0.054 0.007]
 [0.    0.    0.    0.   ]
 [0.017 0.041 0.035 0.046]
 [0.07  0.118 0.106 0.059]
 [0.189 0.176 0.16  0.043]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.088 0.205 0.234 0.176]
 [0.252 0.538 0.527 0.439]
 [0.    0.    0.    0.   ]]
```

Start with random policy

In [37]:
V_pi = np.zeros([nb_states])
policy = np.ones([nb_states, nb_actions]) / nb_actions  # random policy, 25% each action
lmbda = 1.0  # discount

for _ in range(10):
    # flatten MDP
    P_pi, R_pi = flatten_mdp(policy, env.env.P)
    
    # evaluate policy
    for _ in range(100):
        V_pi = R_pi + lmbda * P_pi @ V_pi
        
    # iterate policy
    policy *= 0  # clear
    Q_pi = calc_Q_pi(V_pi, env.env.P, lmbda)
    a_max = np.argmax(Q_pi, axis=-1)     # could distribute action probability between equal max(q) values
    policy[range(nb_states), a_max] = 1  # pick greedy action    

Show optimal policy

In [38]:
a2w = {0:'<', 1:'v', 2:'>', 3:'^'}
policy_arrows = [a2w[x] for x in np.argmax(policy, axis=-1)]
print(np.array(policy_arrows).reshape([-1, 4]))

[['<' '^' '^' '^']
 ['<' '<' '<' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]


Correct optimal policy:
```
[['<' '^' '^' '^']
 ['<' '<' '<' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]
```

# Value Iteration

In [80]:
V_pi = np.zeros([nb_states])
policy = np.ones([nb_states, nb_actions]) / nb_actions  # random policy, 25% each action
lmbda = 1.0  # discount

for i in range(50):
    
    Q_pi = calc_Q_pi(V_pi, env.env.P, lmbda)
    a_max = np.argmax(Q_pi, axis=-1)
    
    V_pi[range(nb_states)] = Q_pi[range(nb_states), a_max]

# construct policy
policy = np.zeros([nb_states, nb_actions])
policy[range(nb_states), a_max] = 1 

Show optimal policy

In [81]:
a2w = {0:'<', 1:'v', 2:'>', 3:'^'}
policy_arrows = [a2w[x] for x in np.argmax(policy, axis=-1)]
print(np.array(policy_arrows).reshape([-1, 4]))

[['<' '^' '^' '^']
 ['<' '<' '<' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]


Correct optimal policy:
```
[['<' '^' '^' '^']
 ['<' '<' '<' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]
```